In [1]:
import os
os.chdir('../src')

In [2]:
import numpy as np
import pandas as pd

from digital_twin.hospital.config_loader import load_hospital_config
from digital_twin.hospital.data_prep import load_patients
from digital_twin.hospital.bed_capacity import run_hospital_multiward_pipeline
from digital_twin.hospital.scenarios import run_scenario, summarize_hospital_runs
from digital_twin.core.hospital_des import run_hospital_des

ENTRY_WARD_DEFAULT = "SEH"
CONFIG_PATH_DEFAULT = "digital_twin/hospital/configs/hospital_1.yml"


In [3]:
def load_baseline_objects(
    config_path: str = CONFIG_PATH_DEFAULT,
    entry_ward: str = ENTRY_WARD_DEFAULT,
):
    cfg = load_hospital_config(config_path)
    patients = load_patients(cfg.simulation["data_dir"])

    res = run_hospital_multiward_pipeline(
        cfg=cfg,
        entry_ward=entry_ward,
        patients=patients,
    )
    cfg = res["cfg"]
    mu_base = res["mu_series"]

    return cfg, patients, mu_base

cfg, patients, mu_base = load_baseline_objects()
cfg


Building flow graph from routing rules:


HospitalConfig(simulation={'data_dir': 'digital_twin/hospital/data', 'horizon_days': 180, 'n_runs_mc': 3000, 'sla_alpha': 0.05, 'random_seed': 42, 'capacity_default': 36, 'service_default': 'surgery'}, meta=HospitalMeta(name='Example Hospital 1', timezone='Europe/Amsterdam', simulation_horizon_days=180), wards={'SEH': Ward(id='SEH', name='Spoedeisende Hulp', type='ED', capacity=12, processtime_model=LOSModel(type='lognormal', params={'mu': 1.0, 'sigma': 0.5}, source=None)), 'ICU': Ward(id='ICU', name='Intensive Care', type='ICU', capacity=10, processtime_model=LOSModel(type='gamma', params={'shape': 2.1, 'scale': 1.3}, source=None)), 'WARD_A': Ward(id='WARD_A', name='Chirurgische afdeling', type='WARD', capacity=24, processtime_model=LOSModel(type='from_data', params={'service': 'surgery'}, source=None)), 'WARD_B': Ward(id='WARD_B', name='Interne afdeling', type='WARD', capacity=20, processtime_model=LOSModel(type='from_data', params={'service': 'general_medicine'}, source=None))}, pat

In [ ]:
def simulate_baseline(cfg, patients, mu_base,
                      entry_ward=ENTRY_WARD_DEFAULT,
                      warmup_days=14,
                      n_rep=200):
    """
    Draait de baseline DES en geeft alle runs terug als DataFrame,
    met een 'scenario' kolom = 'baseline'.
    """
    df = run_hospital_des(
        cfg=cfg,
        entry_ward=entry_ward,
        mu_series=mu_base,
        patients_df=patients,
        warmup_days=warmup_days,
        n_rep=n_rep,
    )
    df = df.copy()
    df["scenario"] = "baseline"
    return df


def simulate_scenario(cfg, patients, mu_base, scenario_dict,
                      entry_ward=ENTRY_WARD_DEFAULT,
                      warmup_days=14,
                      n_rep=200):
    """
    Wrapper om een scenario te draaien met dezelfde engine als in Streamlit.
    """
    df_scen = run_scenario(
        cfg,
        entry_ward,
        patients,
        mu_base,
        scenario_dict,
        warmup_days,
        n_rep,
    )
    df_scen = df_scen.copy()
    df_scen["scenario"] = scenario_dict["name"]
    return df_scen


baseline_df = simulate_baseline(cfg, patients, mu_base)
baseline_df.head()


Building flow graph from routing rules:


,ward_id,beds,rep,mean_wait,p95_wait,max_wait,mean_occupancy,p95_occupancy,days_over_95pct,scenario
0,SEH,12,0,0.0,0.0,0.0,1.700000,5.0,0.0,baseline
1,ICU,10,0,0.0,0.0,0.0,0.000000,0.0,0.0,baseline
2,WARD_A,24,0,0.0,0.0,0.0,0.000000,0.0,0.0,baseline
3,WARD_B,20,0,0.0,0.0,0.0,0.000000,0.0,0.0,baseline
4,SEH,12,1,0.0,0.0,0.0,1.395238,4.0,0.0,baseline


In [ ]:
scenarios = [
    {"name": "baseline", "arrival_scale": 1.0},
    {"name": "demand_250", "arrival_scale": 2.5},
    {"name": "demand_300", "arrival_scale": 3.0},
{
    "name": "demand_300_capacity_boost",
    "arrival_scale": 3.0,
    "capacity_overrides": {
        "SEH": 14,      # +2 beds
        "ICU": 12,      # +2
        "WARD_A": 28,   # +4
        "WARD_B": 24,   # +4
    },
}]


In [17]:
for sc in scenarios[1:]:
    df_s = simulate_scenario(cfg, patients, mu_base, sc)
    summary = summarize_hospital_runs(pd.concat([baseline_df, df_s], ignore_index=True))
    print(sc["name"], summary.groupby("scenario")["days_over_95pct"].sum())


HospitalConfig(simulation={'data_dir': 'digital_twin/hospital/data', 'horizon_days': 180, 'n_runs_mc': 3000, 'sla_alpha': 0.05, 'random_seed': 42, 'capacity_default': 36, 'service_default': 'surgery'}, meta=HospitalMeta(name='Example Hospital 1', timezone='Europe/Amsterdam', simulation_horizon_days=180), wards={'SEH': Ward(id='SEH', name='Spoedeisende Hulp', type='ED', capacity=12, processtime_model=LOSModel(type='lognormal', params={'mu': 1.0, 'sigma': 0.5}, source=None)), 'ICU': Ward(id='ICU', name='Intensive Care', type='ICU', capacity=10, processtime_model=LOSModel(type='gamma', params={'shape': 2.1, 'scale': 1.3}, source=None)), 'WARD_A': Ward(id='WARD_A', name='Chirurgische afdeling', type='WARD', capacity=24, processtime_model=LOSModel(type='from_data', params={'service': 'surgery'}, source=None)), 'WARD_B': Ward(id='WARD_B', name='Interne afdeling', type='WARD', capacity=20, processtime_model=LOSModel(type='from_data', params={'service': 'general_medicine'}, source=None))}, pat

In [ ]:
def compute_scenario_risk_score(merged: pd.DataFrame) -> float:
    """
    Risicoscore 0–100 op basis van:
      - extra dagen >95% bezetting t.o.v. baseline
      - bezettingsgraad in scenario
    Expect:
      mean_occupancy_base / _scen
      days_over_95pct_base / _scen
      capacity
    """
    if merged.empty:
        return 0.0

    m = merged.copy()
    for col in [
        "mean_occupancy_scen",
        "days_over_95pct_scen",
        "days_over_95pct_base",
        "delta_days_over_95pct",
        "capacity",
    ]:
        if col not in m.columns:
            m[col] = 0.0

    ward_scores = []
    for _, row in m.iterrows():
        scen_occ = float(row["mean_occupancy_scen"])
        base_days = float(row["days_over_95pct_base"])
        scen_days = float(row["days_over_95pct_scen"])
        d_days = float(row["delta_days_over_95pct"])
        cap = float(row["capacity"]) if row["capacity"] not in (None, 0) else 0.0

        extra_days = max(d_days, 0.0)
        days_factor = min(extra_days / 10.0, 3.0)

        occ_frac = scen_occ / cap if cap > 0 else 0.0
        occ_factor = max(occ_frac - 0.85, 0.0) / 0.05
        occ_factor = max(0.0, min(occ_factor, 3.0))

        ward_scores.append(days_factor + occ_factor)

    if not ward_scores:
        return 0.0

    avg_score = float(np.mean(ward_scores))  # 0–6
    risk = (avg_score / 6.0) * 100.0
    return max(0.0, min(risk, 100.0))


In [ ]:
def compute_kpis_for_scenario(baseline_df, scen_df, scen_name):
    """
    Bouwt KPI's voor één scenario t.o.v. baseline.
    Gebruikt de echte kolommen uit summarize_hospital_runs:
    mean_wait, p95_wait, mean_occupancy, p95_occupancy, days_over_95pct.
    """
    df_all = pd.concat([baseline_df, scen_df], ignore_index=True)
    summary = summarize_hospital_runs(df_all)

    base = summary[summary["scenario"] == "baseline"].copy()
    scen = summary[summary["scenario"] == scen_name].copy()

    if base.empty or scen.empty:
        raise ValueError("Baseline or scenario summary missing.")

    merged = base.merge(
        scen,
        on="ward_id",
        suffixes=("_base", "_scen"),
    )

    merged["delta_mean_occupancy"] = (
        merged["mean_occupancy_scen"] - merged["mean_occupancy_base"]
    )
    merged["delta_p95_occupancy"] = (
        merged["p95_occupancy_scen"] - merged["p95_occupancy_base"]
    )
    merged["delta_days_over_95pct"] = (
        merged["days_over_95pct_scen"] - merged["days_over_95pct_base"]
    )
    merged["delta_mean_wait"] = (
        merged["mean_wait_scen"] - merged["mean_wait_base"]
    )
    merged["delta_p95_wait"] = (
        merged["p95_wait_scen"] - merged["p95_wait_base"]
    )

    kpi = {
        "scenario": scen_name,
        "mean_occupancy_base": merged["mean_occupancy_base"].mean(),
        "mean_occupancy_scen": merged["mean_occupancy_scen"].mean(),
        "mean_occupancy_delta": merged["delta_mean_occupancy"].mean(),

        "p95_occupancy_base": merged["p95_occupancy_base"].mean(),
        "p95_occupancy_scen": merged["p95_occupancy_scen"].mean(),
        "p95_occupancy_delta": merged["delta_p95_occupancy"].mean(),

        "days_over_95_base": merged["days_over_95pct_base"].sum(),
        "days_over_95_scen": merged["days_over_95pct_scen"].sum(),
        "days_over_95_delta": merged["delta_days_over_95pct"].sum(),

        "mean_wait_base": merged["mean_wait_base"].mean(),
        "mean_wait_scen": merged["mean_wait_scen"].mean(),
        "mean_wait_delta": merged["delta_mean_wait"].mean(),

        "p95_wait_base": merged["p95_wait_base"].mean(),
        "p95_wait_scen": merged["p95_wait_scen"].mean(),
        "p95_wait_delta": merged["delta_p95_wait"].mean(),
    }

    return kpi


In [20]:
baseline_df = simulate_baseline(cfg, patients, mu_base)

kpi_rows = []
for sc in scenarios:
    name = sc["name"]
    if name == "baseline":
        scen_df = baseline_df.copy()
    else:
        scen_df = simulate_scenario(cfg, patients, mu_base, sc)

    kpi_rows.append(compute_kpis_for_scenario(baseline_df, scen_df, name))

kpis = pd.DataFrame(kpi_rows)
kpis


Building flow graph from routing rules:
HospitalConfig(simulation={'data_dir': 'digital_twin/hospital/data', 'horizon_days': 180, 'n_runs_mc': 3000, 'sla_alpha': 0.05, 'random_seed': 42, 'capacity_default': 36, 'service_default': 'surgery'}, meta=HospitalMeta(name='Example Hospital 1', timezone='Europe/Amsterdam', simulation_horizon_days=180), wards={'SEH': Ward(id='SEH', name='Spoedeisende Hulp', type='ED', capacity=12, processtime_model=LOSModel(type='lognormal', params={'mu': 1.0, 'sigma': 0.5}, source=None)), 'ICU': Ward(id='ICU', name='Intensive Care', type='ICU', capacity=10, processtime_model=LOSModel(type='gamma', params={'shape': 2.1, 'scale': 1.3}, source=None)), 'WARD_A': Ward(id='WARD_A', name='Chirurgische afdeling', type='WARD', capacity=24, processtime_model=LOSModel(type='from_data', params={'service': 'surgery'}, source=None)), 'WARD_B': Ward(id='WARD_B', name='Interne afdeling', type='WARD', capacity=20, processtime_model=LOSModel(type='from_data', params={'service': 

,scenario,mean_occupancy_base,mean_occupancy_scen,mean_occupancy_delta,p95_occupancy_base,p95_occupancy_scen,p95_occupancy_delta,days_over_95_base,days_over_95_scen,days_over_95_delta,mean_wait_base,mean_wait_scen,mean_wait_delta,p95_wait_base,p95_wait_scen,p95_wait_delta
0,baseline,0.345631,0.345631,0.000000,1.011,1.011000,0.000000,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,demand_250,0.345631,0.865554,0.519923,1.011,2.040812,1.029812,0.0,1.07,1.07,0.0,0.000640,0.000640,0.0,0.000000,0.000000
2,demand_300,0.345631,1.037071,0.691440,1.011,2.407687,1.396687,0.0,3.66,3.66,0.0,0.002889,0.002889,0.0,0.009129,0.009129
3,demand_300_capacity_boost,0.345631,1.039006,0.693375,1.011,2.379500,1.368500,0.0,0.82,0.82,0.0,0.000486,0.000486,0.0,0.000328,0.000328


In [21]:
kpis.to_csv("capacity_kpis.csv", index=False)
kpis


,scenario,mean_occupancy_base,mean_occupancy_scen,mean_occupancy_delta,p95_occupancy_base,p95_occupancy_scen,p95_occupancy_delta,days_over_95_base,days_over_95_scen,days_over_95_delta,mean_wait_base,mean_wait_scen,mean_wait_delta,p95_wait_base,p95_wait_scen,p95_wait_delta
0,baseline,0.345631,0.345631,0.000000,1.011,1.011000,0.000000,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,demand_250,0.345631,0.865554,0.519923,1.011,2.040812,1.029812,0.0,1.07,1.07,0.0,0.000640,0.000640,0.0,0.000000,0.000000
2,demand_300,0.345631,1.037071,0.691440,1.011,2.407687,1.396687,0.0,3.66,3.66,0.0,0.002889,0.002889,0.0,0.009129,0.009129
3,demand_300_capacity_boost,0.345631,1.039006,0.693375,1.011,2.379500,1.368500,0.0,0.82,0.82,0.0,0.000486,0.000486,0.0,0.000328,0.000328


In [11]:
test_df = summarize_hospital_runs(pd.concat([baseline_df, scen_df], ignore_index=True))
test_df.columns


Index(['scenario', 'ward_id', 'mean_wait', 'p95_wait', 'max_wait',
       'mean_occupancy', 'p95_occupancy', 'days_over_95pct'],
      dtype='object')

In [12]:
simulate_scenario(cfg, patients, mu_base, scenarios[1]).head()


HospitalConfig(simulation={'data_dir': 'digital_twin/hospital/data', 'horizon_days': 180, 'n_runs_mc': 3000, 'sla_alpha': 0.05, 'random_seed': 42, 'capacity_default': 36, 'service_default': 'surgery'}, meta=HospitalMeta(name='Example Hospital 1', timezone='Europe/Amsterdam', simulation_horizon_days=180), wards={'SEH': Ward(id='SEH', name='Spoedeisende Hulp', type='ED', capacity=12, processtime_model=LOSModel(type='lognormal', params={'mu': 1.0, 'sigma': 0.5}, source=None)), 'ICU': Ward(id='ICU', name='Intensive Care', type='ICU', capacity=10, processtime_model=LOSModel(type='gamma', params={'shape': 2.1, 'scale': 1.3}, source=None)), 'WARD_A': Ward(id='WARD_A', name='Chirurgische afdeling', type='WARD', capacity=24, processtime_model=LOSModel(type='from_data', params={'service': 'surgery'}, source=None)), 'WARD_B': Ward(id='WARD_B', name='Interne afdeling', type='WARD', capacity=20, processtime_model=LOSModel(type='from_data', params={'service': 'general_medicine'}, source=None))}, pat

,ward_id,beds,rep,mean_wait,p95_wait,max_wait,mean_occupancy,p95_occupancy,days_over_95pct,scenario
0,SEH,12,0,0.00000,0.0,0.000000,2.619048,6.55,0.0,demand_200
1,ICU,10,0,0.00000,0.0,0.000000,0.000000,0.00,0.0,demand_200
2,WARD_A,24,0,0.00000,0.0,0.000000,0.000000,0.00,0.0,demand_200
3,WARD_B,20,0,0.00000,0.0,0.000000,0.000000,0.00,0.0,demand_200
4,SEH,12,1,0.00072,0.0,0.149849,2.919048,7.00,1.0,demand_200


In [13]:
import pprint
pprint.pprint(scenarios)


[{'arrival_scale': 1.0, 'name': 'baseline'},
 {'arrival_scale': 2.0, 'name': 'demand_200'},
 {'arrival_scale': 2.5, 'name': 'demand_250'},
 {'arrival_scale': 2.5,
  'capacity_overrides': {'WARD_A': 20},
  'name': 'demand_250_capacity'}]
